In [ ]:
"""
 시퀀스투시퀀스Seq2seq,
 내부적으로 모델은 인코더 와 디코더 로 구성됩니다.
 인코더 는 입력 시퀀스의 각 항목을 처리하고 캡처한 정보를 벡터(컨텍스트라고 함 ) 로 컴파일합니다.
 전체 입력 시퀀스를 처리한 후 인코더는 컨텍스트를 디코더로 보내고
 디코더는 항목별로 출력 시퀀스 항목을 생성하기 시작합니다.
 기계 번역의 경우 컨텍스트는 벡터(기본적으로 숫자 배열)입니다. 
 인코더와 디코더는 모두 순환 신경망인 경향이 있습니다
 모델을 설정할 때 컨텍스트 벡터 의 크기를 설정할 수 있습니다.
 기본적으로 인코더 RNN 의 숨겨진 유닛 수입니다 . 이러한 시각화에서는 크기가4인(4차원) 벡터를 표시하지만
 실제 애플리케이션에서 컨텍스트 벡터의 크기는 256, 512 또는 1024와 같습니다.
 설계상 RNN은 각 시간 단계에서 두 개의 입력, 즉 입력(인코더의 경우 입력 문장의 한 단어)과
 숨겨진 상태를 사용합니다. 그러나 단어는 벡터로 표현되어야 합니다. 
 단어를 벡터로 변환하려면 "word embedding" 알고리즘 이라는 메서드 클래스를 사용합니다. 
 이는단어를 단어의 많은 의미/의미론적 정보를 포착하는 벡터공간으로 변환합니다(예:왕 - 남자 + 여자 =여왕 ).
 입력 단어를 처리하기 전에 벡터로 변환해야 합니다. 
 이러한 변환은 단어 임베딩 알고리즘을 사용하여 수행됩니다. 
 사전 훈련된 임베딩을 사용하거나 데이터세트에 자체 임베딩을 훈련 할 수 있습니다. 
 크기가 200 또는 300인 벡터를 임베딩하는 것이 일반적입니다. 단순화를 위해 크기가 4인 벡터를 표시합니다.
 
 Atention 모델은 두 가지 주요 측면에서 기존의 시퀀스-투-시퀀스 모델과 다릅니다.
 첫째, 인코더는 훨씬 더 많은 데이터를 디코더  전달합니다. 
 인코더는 인코딩 단계의 마지막 숨겨진상태(hidden states)를 전달하는 대신 모든 숨겨진 상태를 디코더에 전달합니다.
 둘째, 주의 디코더는 출력을 생성하기 전에 추가 단계를 수행합니다.
 이 디코딩 시간 단계와 관련된 입력 부분에 초점을 맞추기 위해 디코더는 다음 을 수행합니다.
 1.수신된 인코더 hidden state 세트를 살펴보세요 . 각 인코더 hidden states는 입력 문장의 특정 단어와 가장 연관되어 있습니다.
 2.각 hidden state에 점수를 부여합니다(지금은 점수 계산 방법을 무시하겠습니다).
 3.각 hidden state 에 softmaxed 점수를 곱하여 점수가 높은 hidden states를 증폭하고 
 점수가 낮은 hidden states를 제거합니다.
 
 각각의 RNN셀들마다 hidden state가 있다.
 Seq2seq + attention, 번역기에서는 입출력 길이가 다를수 있다
 
16-01 트랜스포머(Transformer)
https://nlpinkorean.github.io/illustrated-transformer/
"""

In [10]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [11]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"ZIP file downloaded to {output_path}")
    else:
        print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

ZIP file downloaded to fra-eng.zip


In [12]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 227815


In [13]:
lines

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
...,...,...
227810,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
227811,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
227812,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
227813,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...


In [14]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
10843,Go look for it.,Va le chercher !
56302,I didn't wait for Tom.,Je n'ai pas attendu Tom.
17815,Tom is insecure.,Tom est incertain.
42851,Please stop talking.,"Cessez de parler, je vous prie !"
29623,This seems stupid.,Ça semble stupide.
54276,Your dog is drooling.,Ton chien bave.
12126,It looks cheap.,Cela semble bon marché.
47197,Have a look at these.,Jetez un œil à celles-ci.
5566,I'm doing OK.,Je vais bien.
17332,The food's here.,La nourriture est arrivée.


In [15]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')#\t가 시작,\n가 종료 심볼
lines.sample(10)

,src,tar
13927,You lied to me.,\t Tu me mentis. \n
33354,I follow the rules.,\t Je suis les règles. \n
11972,I'm the oldest.,\t Je suis le plus vieux. \n
26549,I felt very happy.,\t Je me sentais très heureuse. \n
27182,I started the car.,\t J'ai démarré la voiture. \n
21870,It's a rainy day.,\t C'est jour pluvieux. \n
44970,We're not prisoners.,\t Nous ne sommes pas prisonnières. \n
19150,Are you a wizard?,\t Es-tu une sorcière ? \n
12853,The baby cried.,\t Le bébé a pleuré. \n
24252,What woke you up?,\t Qu'est-ce qui vous a réveillées ? \n


In [16]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

In [17]:
src_vocab

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ï',
 '’',
 '€'}

In [18]:
tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [19]:
tar_vocab

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f',
 '‽'}

In [20]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 80
target 문장의 char 집합 : 104


In [23]:
src_vocab=sorted(list(src_vocab))

In [24]:
src_vocab

[' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ï',
 '’',
 '€']

In [25]:
tar_vocab = sorted(list(tar_vocab))

In [26]:
tar_vocab

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f',
 '‽']

In [27]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, 'ï': 77, '’': 78, '€': 79}


In [30]:
print(tar_to_index)

{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 37, 'L': 38, 'M': 39, 'N': 40, 'O': 41, 'P': 42, 'Q': 43, 'R': 44, 'S': 45, 'T': 46, 'U': 47, 'V': 48, 'W': 49, 'X': 50, 'Y': 51, 'a': 52, 'b': 53, 'c': 54, 'd': 55, 'e': 56, 'f': 57, 'g': 58, 'h': 59, 'i': 60, 'j': 61, 'k': 62, 'l': 63, 'm': 64, 'n': 65, 'o': 66, 'p': 67, 'q': 68, 'r': 69, 's': 70, 't': 71, 'u': 72, 'v': 73, 'w': 74, 'x': 75, 'y': 76, 'z': 77, '\xa0': 78, '«': 79, '»': 80, 'À': 81, 'Ç': 82, 'É': 83, 'Ê': 84, 'Ô': 85, 'à': 86, 'â': 87, 'ç': 88, 'è': 89, 'é': 90, 'ê': 91, 'ë': 92, 'î': 93, 'ï': 94, 'ô': 95, 'ù': 96, 'û': 97, 'œ': 98, '\u2009': 99, '‘': 100, '’': 101, '\u202f': 102, '‽': 103}


In [28]:
encoder_input = [] #[30,64],[], . .., []

In [31]:
for line in lines.src:
    encoded_line=[]
    for c in line:
        encoded_line.append(src_to_index[c])
    encoder_input.append(encoded_line)

In [33]:
encoder_input

[[30, 64, 10],
 [30, 64, 10],
 [30, 64, 10],
 [30, 64, 10],
 [31, 58, 10],
 [31, 58, 10],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [46, 57, 64, 23],
 [46, 64, 72, 2],
 [46, 64, 72, 2],
 [46, 64, 72, 2],
 [27, 70, 52, 60, 2],
 [27, 70, 52, 60, 2],
 [27, 70, 52, 60, 2],
 [29, 58, 67, 54, 2],
 [31, 54, 61, 65, 2],
 [31, 58, 53, 54, 10],
 [31, 58, 53, 54, 10],
 [33, 70, 62, 65, 2],
 [33, 70, 62, 65, 10],
 [42, 69, 64, 65, 2],
 [42, 69, 64, 65, 2],
 [42, 69, 64, 65, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [25, 54, 56, 58, 63, 10],
 [25, 54, 56, 58, 63, 10],
 [30, 64, 1, 64, 63, 10],
 [30, 64, 1, 64, 63, 10],
 [30, 64, 

In [34]:
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [1, 3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [1, 3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [1, 3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [35]:
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 52, 3, 4, 3, 2], [3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [36]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [37]:
#패딩해준것
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [38]:
#원핫인코딩
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [40]:
encoder_input.shape #60000건, 최대길이 22, 

(60000, 22, 80)

In [42]:
decoder_input.shape

(60000, 76, 104)

In [41]:
decoder_target.shape

(60000, 76, 104)

In [43]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [51]:
encoder_inputs = Input(shape=(None, src_vocab_size))#src_vocab_size 80 => 문자 벡터의 크기

In [52]:
src_vocab_size

80

In [53]:
encoder_lstm = LSTM(units=256, return_state=True)

In [54]:
encoder_outputs, state_h, state_c =encoder_lstm(encoder_inputs)
#state_h(은닉상태), state_c(셀상태)

In [55]:
encoder_states = [state_h, state_c] #encoder_states: 컨텍스트 벡터

In [56]:
#Seq2seq 디코더 설계

In [57]:
decoder_inputs = Input(shape=(None, tar_vocab_size))

In [58]:
decoder_lstm = LSTM(units=256, return_state=True, return_sequences=True)

In [62]:
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
#디코더는 은닉 상태, 셀 상태 정보가 필요 없음(훈련과정에서는) _(언더바)처리

In [63]:
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')

In [64]:
decoder_outputs=decoder_softmax_layer(decoder_outputs)

In [65]:
model=Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [70]:
model.compile(optimizer='rmsprop', loss="categorical_crossentropy")

In [72]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 [==============================] - 119s 155ms/step - loss: 0.8441 - val_loss: 0.7744
Epoch 2/40
750/750 [==============================] - 125s 166ms/step - loss: 0.5702 - val_loss: 0.6600
Epoch 3/40
750/750 [==============================] - 125s 167ms/step - loss: 0.5020 - val_loss: 0.6017
Epoch 4/40
750/750 [==============================] - 126s 168ms/step - loss: 0.4562 - val_loss: 0.5618
Epoch 5/40
750/750 [==============================] - 125s 167ms/step - loss: 0.4220 - val_loss: 0.5227
Epoch 6/40
750/750 [==============================] - 120s 160ms/step - loss: 0.3959 - val_loss: 0.4978
Epoch 7/40
750/750 [==============================] - 116s 155ms/step - loss: 0.3753 - val_loss: 0.4732
Epoch 8/40
750/750 [==============================] - 115s 154ms/step - loss: 0.3590 - val_loss: 0.4568
Epoch 9/40
750/750 [==============================] - 117s 156ms/step - loss: 0.3453 - val_loss: 0.4453
Epoch 10/40
750/750 [==============================] - 115s 153m

In [ ]:
# lstm에서는 출력이 두개다 히든스테이트정보와 셀스테이트정보

In [ ]:
# 테스트 과정
1. 번역 대상 입력 문장이 인코더에 들어감 -> 은닉/셀 상태 -> 디코더
2. 디코더의 입력 시그널이 전달(sos, \t)
3. 디코더는 다음 문자 예측

In [73]:
# 테스트 과정에서의 디코더 정의

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [74]:
#테스트 과정에서의 인코더 정의
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [75]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [87]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
  # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ""
  # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
    # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char
    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_tar_len):
            stop_condition = True
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.
        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]
        return decoded_sentence

In [88]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 17ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: 
1/1 [==============================] - 0s 17ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: 
1/1 [==============================] - 0s 17ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: 
1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: 
1/1 [==============================] - 0s 17ms/step
-----------------------------------
입력 문장: Get going.
정답 문장: En avant. 
번역 문장: 
